# Comparación con el resto del mundo
> Como vienen evolucionando los paises donde COVID-19 llego antes en comparacion con Argentina?

- comments: false
- author: Pablo Zivic
- categories: [graficos]
- image: images/argentina-en-contexto.png
- permalink: /argentina-en-contexto/

In [1]:
#hide

from io import StringIO
import csv
import pandas as pd
from datetime import datetime
import requests

BASE_URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'

def parse_data(fname):
    csv_content = requests.get(BASE_URL + fname).content
    docs = list(csv.DictReader(StringIO(csv_content.decode('utf8'))))
        
    new_docs = []
    for doc in docs:
        meta = {k: doc[k] for k in ['Province/State', 'Country/Region', 'Lat', 'Long']}
        for k, v in doc.items():
            if k in meta: continue
            new_doc = meta.copy()
            new_doc['date'] = datetime.strptime(k, '%m/%d/%y')
            if not v: continue
            new_doc['cnt'] = int(v)
            new_docs.append(new_doc)


    return (
        pd.DataFrame(new_docs)
          .rename(
              columns={
                  'Province/State': 'province',
                  'Country/Region': 'country',
                  'Lat': 'lat', 'Long': 'long'
              }
          ).groupby(['date', 'country'])
          .cnt.sum()
          .reset_index()
    )

In [2]:
#hide
confirmed_df = parse_data('csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
confirmed_df = confirmed_df.rename(columns=dict(cnt='confirmed'))

In [3]:
#hide
import numpy as np

dfs = []
for c in confirmed_df.country.unique():
    cdf = confirmed_df[confirmed_df.country == c].set_index('date').sort_index()
    cdf['rate'] = np.minimum(1.6, cdf.confirmed / cdf.confirmed.shift(1)).rolling(7).mean().fillna(1.6)
    dfs.append(cdf)
    
confirmed_df = pd.concat(dfs).reset_index()

In [4]:
#hide
cnt_by_country = confirmed_df.groupby('country').confirmed.max().sort_values(ascending=False)
rate_by_country = confirmed_df[confirmed_df.confirmed > 1000].groupby('country').rate.max().sort_values(ascending=False)

In [5]:
#hide
import altair as alt

def plot_rate_cnt_trajectories(countries, selected):
    selection = alt.selection_multi(
        fields=['country'], bind='legend', init=[{'country': c} for c in selected]
    )
    
    source = confirmed_df[(confirmed_df.confirmed > 30) & (confirmed_df.country.isin(countries))]
    
    return (
        alt.Chart(source)
           .mark_line(point=True, radius=150)
           .encode(
               x=alt.X('confirmed', scale=alt.Scale(type='log'), axis=alt.Axis(title='Confirmed Cases')),
               y=alt.Y('rate', axis=alt.Axis(title='Rate of growth'), scale=alt.Scale(domain=[0.95, 1.65])),
               color=alt.Color('country:N', sort=selected),
               tooltip=list(confirmed_df.columns), 
               opacity=alt.condition(selection, alt.value(.8), alt.value(.05))
            ).add_selection(selection)
             .configure_point(size=200).properties(width=650, height=400)
    )

## Argentina comparado con el resto del mundo

**Shift + Click** sobre la leyenda para agregar o quitar paises

In [6]:
#hide_input
latam = [
    'Brazil', 'Chile', 'Argentina', 'Panama', 'Colombia','Mexico', 
    'Ecuador', 'Costa Rica', 'Venezuela', 'Dominican Republic', 'Bolivia',
    'Paraguay', 'Uruguay','Honduras', 'Cuba', 'Puerto Rico','Guatemala', 
]

with_most_cases = list(cnt_by_country.index[:7])
controlled_situations = list(rate_by_country[rate_by_country < 1.1].index)

plot_rate_cnt_trajectories(
    with_most_cases + controlled_situations + latam, 
    selected=['Argentina'] + list(rate_by_country[rate_by_country < 1.05].index) + with_most_cases[:3]
)

alt.Chart(...)

- Eje X: Cantidad de casos confirmados
- Eje Y: Tasa diaria de crecimiento

# Mirando al resto de latino america

Comparando con algunos paises de fuera de latam

In [7]:
#hide_input
latam = [
    'Brazil', 'Chile', 'Argentina', 'Panama', 'Colombia','Mexico', 
    'Ecuador', 'Costa Rica', 'Venezuela', 'Dominican Republic', 'Bolivia',
    'Paraguay', 'Uruguay','Honduras', 'Cuba', 'Puerto Rico','Guatemala', 'Japan', 'US'
]

plot_rate_cnt_trajectories(latam, ['Argentina', 'Chile', 'Brazil', 'Ecuador', 'Japan', 'US'])

alt.Chart(...)

Datos obtenidos de https://github.com/CSSEGISandData/COVID-19